# TM10007 Assignment template

In [6]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/Machine-Learning-TM10007-G9/Lipo-MRI.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 7

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [7]:
# Data loading functions. Uncomment the one you want to use
#from worcgist.load_data import load_data
from worclipo.load_data import load_data
#from worcliver.load_data import load_data
#from ecg.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 115
The number of columns: 494


# **Import important functions**

In [8]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn


# Classifiers
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition

# Create X and **Y**

In [18]:
x = data.iloc[:, 1:].values
y = np.array(data['label'])

In [19]:
print(x)
print(y)

[[3.68433535e-01 2.62371019e-01 2.62003946e+01 ... 3.23830499e+03
  3.15148298e-01 1.44623614e+01]
 [6.60570848e-01 1.79012554e-01 5.51550462e+01 ... 1.53647244e+04
  2.51255317e-01 1.71702745e+01]
 [3.65191450e-01 1.57712493e-01 3.05770259e+01 ... 3.77179816e+03
  3.69366254e-01 1.43948122e+01]
 ...
 [3.07562053e-01 1.69252018e-01 4.94737956e+01 ... 4.96797676e+03
  2.46028326e-01 1.51067769e+01]
 [5.77333471e-01 2.59572405e-01 3.85229179e+01 ... 2.34864119e+03
  1.61139764e-01 1.47716419e+01]
 [6.35281625e-01 1.45088086e-01 9.98527053e+01 ... 1.04654714e+04
  0.00000000e+00 1.75445559e+01]]
['liposarcoma' 'liposarcoma' 'lipoma' 'liposarcoma' 'lipoma' 'lipoma'
 'lipoma' 'lipoma' 'liposarcoma' 'lipoma' 'lipoma' 'liposarcoma'
 'liposarcoma' 'lipoma' 'liposarcoma' 'liposarcoma' 'lipoma' 'lipoma'
 'lipoma' 'liposarcoma' 'lipoma' 'liposarcoma' 'lipoma' 'liposarcoma'
 'lipoma' 'liposarcoma' 'liposarcoma' 'liposarcoma' 'lipoma' 'liposarcoma'
 'liposarcoma' 'lipoma' 'lipoma' 'lipoma' 'lipoma'

# **Split and scale the data**

In [23]:
# Split data
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, stratify=y)

print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

# Scale the data to be normal
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

92
23
92
23
